In [27]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [28]:
from google.colab import files

uploaded = files.upload()

Saving loan_data.xlsx to loan_data (2).xlsx


In [29]:
# upload an in-built Python (OK semi-in-built) dataset
import pandas as pd
import numpy as np

df = pd.read_excel('loan_data.xlsx')

In [30]:
target = df.pop('loan_is_bad')
features = df
tf_dataset = tf.data.Dataset.from_tensor_slices((features.values, target.values))


In [31]:
for feature, label in tf_dataset.take(5):
    print(feature.numpy(), label.numpy())

[1.12000000e+04 3.60000000e+01 6.62000000e+00 3.43890000e+02
 1.00000000e+00 4.00000000e+00 1.08000000e+05 1.00000000e+00
 2.00000000e+00 2.00000000e+00 1.25200000e+01 0.00000000e+00
 0.00000000e+00 9.00000000e+00 0.00000000e+00 3.78220000e+04
 2.10000000e+01 1.16760600e+04 1.05045300e+04 1.17153000e+03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.43890000e+02
 0.00000000e+00 0.00000000e+00 1.24047306e+02 9.69776610e+01
 9.69776610e+01] 0
[1.00000000e+04 3.60000000e+01 1.11400000e+01 3.28060000e+02
 2.00000000e+00 1.00000000e+00 6.50000000e+04 1.00000000e+00
 1.00000000e+00 2.00000000e+00 9.58000000e+00 0.00000000e+00
 0.00000000e+00 9.00000000e+00 0.00000000e+00 1.66230000e+04
 1.10000000e+01 4.62034000e+03 2.71084000e+03 8.97600000e+02
 0.00000000e+00 1.01190000e+03 1.01190000e+01 3.28060000e+02
 0.00000000e+00 0.00000000e+00 1.55059133e+02 1.19940867e+02
 1.19940867e+02] 1
[8.00000000e+03 3.60000000e+01 1.62900000e+01 2.82410000e+02
 3.00000000e+00 5.00000000e+00 3.50000000e+04 1

In [69]:
n = len(list(tf_dataset))
train_size = int(n * 0.8)
train_dataset = tf_dataset.take(train_size)
test_dataset = tf_dataset.skip(train_size)

In [62]:
train_batch = train_dataset.batch(50)
features, labels = next(iter(train_batch))
# print the first 20 records (rows) of features and labels
print(features[0:20])
print(labels[0:20])

tf.Tensor(
[[1.12000000e+04 3.60000000e+01 6.62000000e+00 3.43890000e+02
  1.00000000e+00 4.00000000e+00 1.08000000e+05 1.00000000e+00
  2.00000000e+00 2.00000000e+00 1.25200000e+01 0.00000000e+00
  0.00000000e+00 9.00000000e+00 0.00000000e+00 3.78220000e+04
  2.10000000e+01 1.16760600e+04 1.05045300e+04 1.17153000e+03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 3.43890000e+02
  0.00000000e+00 0.00000000e+00 1.24047306e+02 9.69776610e+01
  9.69776610e+01]
 [1.00000000e+04 3.60000000e+01 1.11400000e+01 3.28060000e+02
  2.00000000e+00 1.00000000e+00 6.50000000e+04 1.00000000e+00
  1.00000000e+00 2.00000000e+00 9.58000000e+00 0.00000000e+00
  0.00000000e+00 9.00000000e+00 0.00000000e+00 1.66230000e+04
  1.10000000e+01 4.62034000e+03 2.71084000e+03 8.97600000e+02
  0.00000000e+00 1.01190000e+03 1.01190000e+01 3.28060000e+02
  0.00000000e+00 0.00000000e+00 1.55059133e+02 1.19940867e+02
  1.19940867e+02]
 [8.00000000e+03 3.60000000e+01 1.62900000e+01 2.82410000e+02
  3.00000000e+00 5.0000

In [63]:
feature_count = features.shape[1]
print(feature_count)

29


In [64]:
feature_count = features.shape[1]#determine number of features
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(feature_count,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])#dropout rate could be 0.2-0.5 and it is for reducing overfitting

In [65]:
# compile the model
model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001,  weight_decay=0.005),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])#learning_rate and weight_decay are for reducing overfitting(idk whether it is overlapped with dropout rate)
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 64)                1920      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 1)                 33        
                                                                 
Total params: 4033 (15.75 KB)
Trainable params: 4033 (15.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [66]:
history = model.fit(train_batch, epochs=10)

Epoch 1/10
800/800 [==============================] - 5s 4ms/step - loss: 353.5141 - accuracy: 0.8622
Epoch 2/10
800/800 [==============================] - 3s 3ms/step - loss: 38.7487 - accuracy: 0.9138
Epoch 3/10
800/800 [==============================] - 4s 4ms/step - loss: 11.5579 - accuracy: 0.9096
Epoch 4/10
800/800 [==============================] - 3s 3ms/step - loss: 4.4057 - accuracy: 0.9061
Epoch 5/10
800/800 [==============================] - 3s 4ms/step - loss: 2.3638 - accuracy: 0.9321
Epoch 6/10
800/800 [==============================] - 4s 5ms/step - loss: 0.5638 - accuracy: 0.9380
Epoch 7/10
800/800 [==============================] - 3s 3ms/step - loss: 0.5984 - accuracy: 0.9342
Epoch 8/10
800/800 [==============================] - 3s 3ms/step - loss: 0.4010 - accuracy: 0.9423
Epoch 9/10
800/800 [==============================] - 3s 3ms/step - loss: 0.3319 - accuracy: 0.9286
Epoch 10/10
800/800 [==============================] - 3s 4ms/step - loss: 0.3174 - accuracy: 0.

In [70]:
test_batch = test_dataset.batch(32)
test_loss, test_accuracy = model.evaluate(test_batch, verbose=2)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

313/313 - 1s - loss: 0.1860 - accuracy: 0.9471 - 1s/epoch - 3ms/step
Test Loss: 0.18604058027267456, Test Accuracy: 0.9470846652984619


In [75]:
predictions = model.predict(test_batch)
true_labels = np.array([label.numpy() for _, label in test_dataset])
for pred, real in zip(predictions, true_labels):
    predicted_label = (pred > 0.5).astype(int)
    print(f"Predicted: {predicted_label};    Real: {real}")
test_loss, test_accuracy = model.evaluate(test_batch, verbose=2)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

流式输出内容被截断，只能显示最后 5000 行内容。
Predicted: [1];    Real: 1
Predicted: [0];    Real: 0
Predicted: [1];    Real: 1
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [1];    Real: 1
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [1];    Real: 1
Predicted: [1];    Real: 1
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
P